# Tutorial: Combinators (Stateful `evaluate(env)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/combinator.html

This is the stateful variant: definitions are kept in a shared environment across cells.


In [ ]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import EvaluationEnvironment, evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")

env = EvaluationEnvironment()
stateful = evaluate(env)


## Before You Start

1. Run the setup cell first to create `env` and `stateful`.
2. Run cells in order from top to bottom, because later cells may depend on earlier definitions.
3. Each `stateful("...")` call updates and reuses the same environment.
4. If results look odd after reruns, restart the kernel and run all cells again.
5. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## What's a combinator?

Combinators build new functions from existing ones without introducing explicit variables.


In [ ]:
stateful('|∘- 6')


In [ ]:
stateful('7 |∘- 9')


In [ ]:
stateful('14‿8 |∘- 19‿6')
stateful('14‿8 +´∘|∘- 19‿6')


## Comparisons

Use these examples to see scalar and elementwise boolean comparisons.


In [ ]:
stateful('3 < 4')
stateful('4 > ∞')
stateful('∞ < @')


In [ ]:
stateful('\'e\' = "George Boole"')
stateful('+´ \'e\' = "George Boole"')
stateful('\'e\' +´∘= "George Boole"  # With a combinator')


In [ ]:
stateful('"abcd" ×´∘= "abdd"')


In [ ]:
stateful('"abcd" ≡ "abdd"')
stateful('"abc"‿"de" ≡ "abc"‿"de"')


In [ ]:
stateful('2‿3‿4‿2 ≠ 3‿3‿2‿2')
stateful('2‿3‿4‿2 ≢ 3‿3‿2‿2')


## Length, rank, and depth

These primitives inspect structure: item count, dimensionality, and nesting depth.


In [ ]:
stateful('≠ "testing"')
stateful('≠ ⟨⟩')
stateful('≠ ⟨ π, ∘, "element" ⋄ ⟨\'l\',1,5,\'t\'⟩ ⟩')
stateful('≠ 4')


In [ ]:
stateful('= 0.5')
stateful('= ↕3')
stateful("= 'a'")
stateful('= "a"')


In [ ]:
stateful('≡ "dream"                  # An ordinary dream')
stateful('≡ "d"‿"r"‿"e"‿"a"‿"m"      # What if the letters were strings?')
stateful('≡ ⟨ "d"‿"r"‿"e"‿"a"‿"m" ⟩  # We have to go deeper')


## Composition

Composition combines smaller transforms into larger reusable pipelines.


In [ ]:
stateful('(≠"string") = ≠"sting"')


In [ ]:
stateful('=´≠¨ ⟨"string","sting"⟩')


In [ ]:
stateful('"string" =○≠ "sting"')


In [ ]:
stateful('"string" ⋈○≠ "sting"')
stateful('⋈○≠ "sting"')


In [ ]:
stateful('2 ⋆⟜- 3')
stateful('2 ⋆⊸- 3')


In [ ]:
stateful('4 -⊸⌽ " before"  # Rotate to the right by four')
stateful('4 ⌽⁼  " before"  # Okay this time Undo is better')


In [ ]:
stateful('¬⊸× 0.5')


In [ ]:
stateful('↕⊸÷ 8')
stateful('¬⊸× ↕⊸÷ 8')


In [ ]:
stateful('1⊸+ 5')
stateful('+⟜1 5')


In [ ]:
stateful('"const"˜ 5')
stateful('@ "const"˜ 6')


In [ ]:
stateful('+⊸1 5')


In [ ]:
stateful('↕⊸÷ 8')
stateful('(↕8) ÷ 7')


In [ ]:
stateful('-⟜1 8')


In [ ]:
stateful('↕⊸÷⟜(-⟜1) 8')


In [ ]:
stateful('(↕÷-⟜1) 8')


## Base decoding continued

This revisits decoding and progressively compresses it into tacit combinator form.


In [ ]:
stateful('@ + +´¨ (⌽2⋆↕8)⊸×¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


In [ ]:
stateful('@⊸+ +´¨ (⌽2⋆↕8)⊸×¨ -⟜\'0\' "01000010"‿"01010001"‿"01001110"')
stateful('(@⊸+)∘(+´¨)∘((⌽2⋆↕8)⊸×¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
stateful('@⊸+∘(+´∘((⌽2⋆↕8)⊸×)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
stateful('@⊸+∘(+´∘×⟜(⌽2⋆↕8)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
stateful('(((((1×10)+2)×10)+3)×10)+4')
stateful('((1×⟜10⊸+2)×⟜10⊸+3)×⟜10⊸+4   # Make the combining step a function')
stateful('4+⟜(10⊸×)3+⟜(10⊸×)2+⟜(10⊸×)1 # Flip the combining function around')
stateful("+⟜(10⊸×)´ 4‿3‿2‿1            # Now it's a BQN fold")
stateful('+⟜(10⊸×)´ ⌽ 1‿2‿3‿4          # To fold in reverse, reverse then fold')


In [ ]:
stateful('+´∘×⟜(⌽2⋆↕8) "01010001"-\'0\'')
stateful('+⟜(2⊸×)´∘⌽ "01010001"-\'0\'')


In [ ]:
stateful('@⊸+∘(+⟜(2⊸×)´∘⌽¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
stateful('(@+ ·+⟜(2⊸×)´∘⌽¨ -⟜\'0\') "01000010"‿"01010001"‿"01001110"')


## Summary

If any result is unclear, rerun the section above step by step before moving on.
